In [88]:
import pandas as pd
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB, MultinomialNB, BernoulliNB
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score



In [89]:
df = pd.read_csv('data.csv', sep=';')

In [90]:
# filter only interesting data and seperate into features and labels
all_columns = ['Buchungstext', 'Verwendungszweck', 'Beguenstigter/Zahlungspflichtiger',
         'Kontonummer', 'BLZ', 'Betrag']
columns_of_interest = ['Buchungstext', 'BLZ', 'Betrag']
features = df[columns_of_interest]
labels = df['label']

In [91]:
le = preprocessing.LabelEncoder()
for col_name in all_columns:
    print(max(le.fit_transform(df[col_name])))

13
124
58
53
40
93


In [92]:
features = pd.get_dummies(features)
X_train, X_test, y_train, y_test = train_test_split(features, labels, random_state=1)


In [93]:
gnb = GaussianNB()
gnb.fit(X_train, y_train)
y_pred = gnb.predict(X_test)
print('Accuracy score: ', format(accuracy_score(y_test, y_pred)))
print('Precision score: ', format(precision_score(y_test, y_pred, average='macro')))
print('Recall score: ', format(recall_score(y_test, y_pred, average='macro')))
print('F1 score: ', format(f1_score(y_test, y_pred, average='macro')))

Accuracy score:  0.9433962264150944
Precision score:  0.9315476190476191
Recall score:  0.9615384615384616
F1 score:  0.9393719806763285
